# Grade Headnotes

We are given a CSV file of 7.1k Indian Supreme Court court judgements and their headnotes/summaries. We have created a rubric to analyze the headnotes and grade them. We will perform in-context learning by prompting Meta's Llama 3.1 8B model (4-bit quantized) with the rubric and one-shot examples. We want the model to grade the headnotes and provide an explanation for each grade assigned. We will then output everything into another CSV file that we can use for further work.

## Config and Imports

In [2]:
from huggingface_hub import login
from google.colab import userdata
from tqdm.auto import tqdm
from transformers import AutoTokenizer, BitsAndBytesConfig, pipeline
import csv
import numpy as np
import os
import pandas as pd
import torch

In [17]:
MODEL_NAME = "meta-llama/Meta-Llama-3.1-8B-Instruct"
OUTPUT_FILE = 'judgements_and_graded_headnotes.csv'
INPUT_FILE = 'combined_data.csv'
CSV_CHUNK_SIZE = 50

In [4]:
login(userdata.get('HF_TOKEN'))

In [5]:
EXAMPLE_JUDGEMENT_1 = """
minal Appeal No. 146 of 1954.
Appeal from the judgment and order dated the 27th November 1954 of the Court of Judicial Com 683 missioner at Ajmer in Criminal Appeal No. 15 of 1954 arising out of the judgment and order dated 25th August 1954 of the Court of Special Judge at Ajmer in Criminal Case No. 5 of 1953.
B. P. Maheshwari, for the appellant.
C. K. Daphtary, Solicitor General for India, Porus A. Mehta, H. R. Khanna and B. H. Dhebar, for the respondent.
September 21.
The Judgment of the Court was delivered by BHAGWATI J.
This Appeal with a certificate of fitness under article 134(1)(c) of the Constitution against the decision of the Judicial Commissioner at Ajmer raises an important question as to the connotation of the word "officer" contained in section 21(9), Indian Penal Code.
The appellant was a Class III servant employed as a metal examiner, also called Chaser, in the Railway Carriage Workshops at Ajmer.
I He was charged under section 161, Indian Penal Code with having accepted from one Nanak Singh currency notes of the value of Rs. 150 as illegal gratification as a motive for securing a job for one Kallu.
He was also charged under section 5(1)(d) of Act II of 1947 with abusing his position as a public servant and obtaining for himself by corrupt or illegal means pecuniary advantage in the shape of Rs. 150 from the said Nanak Singh.
He was further charged with having committed an offence under section 420, Indian Penal Code for having induced the said Nanak Singh to deliver to him currency notes of the value of Rs. 150 by dishonest representation that be could secure a job for the said Kallu.
The learned Special Judge, State of Ajmer, who tried him in the first instance for the said offences convicted him of the offence under section 161, 'Indian Penal Code as also the offence under section 5(1) (d) of Act II of 1947 and sentenced him to suffer rigorous imprisonment for six months and one year respectively in regard to the same, both the sentences to run concurrently.
In so far, 684 however, as it was not proved that the appellant did not believe when he accepted the money that he could secure or would try to secure a job for Kallu, it was held that no case under section 420, Indian Penal Code was made out and he was acquitted of that charge.
The appeal taken to the Judicial Commissioner, State of Ajmer, by the appellant failed and on the 10th December, 1954, the learned Judicial Commissioner granted to the appellant a certificate of fitness for appeal on two main grounds, viz., (1) whether the appellant was an "officer" within the meaning of clause (9) of section 21, Indian Penal Code, and (2) whether the provisions of section 137 of the Railways Act excluded all railway servants from the definition of public servants except for purposes of Chapter ' IX, Indian Penal Code.
Concurrent findings of fact were reached by both the Courts below on the question as to whether the appellant accepted Rs. 150 from Nanak Singh as and by way of illegal gratification and these findings could not be and were not challenged before us by the learned counsel for the appellant.
The only questions which were canvassed before us were the two legal points which formed the basis of the certificate of fitness for appeal granted by the learned Judicial Commissioner, State of Ajmer, to the appellant.
The second question has now become academical in the facts of the present case by virtue of a decision of this Court in Ram Krishan vs Delhi State(1), which lays down that before the amendment of section 137 of the Railways Act by Act 17 of 1955 railway servants were treated as public servants only for the purposes of Chapter IX, Indian Penal Code, but in any event they were public servants under the Prevention of Corruption Act (Act II of 1947).
In so far as the appellant has, in our opinion, been rightly convicted of the offense under section 5(1)(d) of Act If of 1947 and awarded the sentence of rigorous imprigonment for one year, the question whether he was rightly convicted of the offence under section 161, (1) ; 685.
Indian Penal Code for which he was awarded the lesser sentence of six months ' rigorous imprisonment has become merely academical and the only question which remains to be considered by us here is whether he was an "officer" within the meaning of section 21 (9), Indian Penal Code.
The provisions of law in regard to the first question may be conveniently set out at this stage: Section 2 of the Prevention of Corruption Act II of 1947 provides: "for the purposes of this Act "public servant" means a public servant as defined in section 21 of the Indian Penal Code".
Section 21, Indian Penal Code provides so far as is relevant for the purposes of the present appeal: "The words 'public servant ', denote a person falling under any of the descriptions hereinafter following, namely, Ninth. . . . and every officer in the service or pay of the Government or remunerated by fees or commission for the performance of any public duty".
There is no doubt that the appellant was.
in the service or pay of the Government and was performing the duty of a metal examiner known as Chaser in the Railway Carriage Workshops at Ajmer and was thus performing a public duty.
It was, however, contended that the appellant was not an officer within the meaning of that term as used in section 21(9), Indian Penal Code.
An Officer, it was contended, on the authority of Reg.
vs Ramajirav jivbajirav and another(1), meant one to whom was delegated by the supreme authority some portion of its regulating and coercive powers and who was appointed to represent the State in its relations to individual subjects.
According to the dictum of West, J., the word "officer" meant some person employed to exercise to some extent and in certain circumstances a delegated function of Government.
He was either himself armed with some authority or repre (1) XII Bom.
H.C.R. 1. 386 sentative character or his duties were immediately auxiliary to those of some one who was so armed.
It was, therefore, contended that the appellant being a metal examiner known as Chaser in the Railway Carriage Workshops had not delegated to him by the supreme authority some portion of its regulating and coercive powers nor was he appointed to represent the State in its relations to individual subjects.
He was neither armed with some authority or representative character nor were his duties immediately auxiliary to those of some one who was so armed.
He was not employed to exercise to some extent and in certain circumstances the delegated function of Government and, therefore, was not an "officer" within the meaning of that term as used in section 21(9), Indian Penal Code.
If he was thus not an officer of the Government, he could not be a public servant within the meaning of section 21, Indian Penal Code nor could he be a public servant for the purposes of Act 11 of 1947 and could not be convicted of the offence under section 5(1)(d) of Act II of 1947.
It has to be noted, however, that the case before the learned Judges of the High Court of Bombay in 12 Bombay High Court Reports 1, concerned an Izaphatdar, that is a lessee, of a village who bad undertaken to keep an account of its forest revenue and pay a certain proportion to the Government, keeping the remainder for himself and the question 'that arose for the consideration of the Court was whether such a person was an officer within the meaning of section 21(9), Indian Penal Code.
It was in this context that the aforesaid observations were made by West, J., and the Court came to the conclusion that Deshmukhs and Deshpandes would be sufficiently within the meaning of the clause they being appointed to perform for the State a portion of its functions or to aid those who were its active representatives but not so an Izaphatdar or the lessee such as the accused.
He was not an officer but a mere contractor bound by his engagement but not by the terms of his office or employment to pay a certain proportion to the Government.
There was no delegation to him of any 687 authority for coercion or interference nor was he an assistant appointed to help any one who was vested with such authority.
The duties which he performed were contractual duties frauduler deception in the discharge of which might subject him to punishment for cheating but not duties attached to any office conferred on him or his predecessor in title, failure to perform which with integrity could make him liable, as an officer, to the special penalties prescribed for delinquent public servants.
This decision in 12 Bombay High Court Reports 1, came to be considered by the Calcutta High Court in Nazamuddin vs Queen Empress(1).
The petitioner in that case was a peon attached to the office of the Superintendent of the Salt Department in the district of Mozafferpur and he had been convicted under section 161, Indian Penal Code.
The contention urged on behalf of the petitioner was that he did not fall within the terms of the last portion of clause (9) of section 21, Indian Penal Code which declared "every officer in the service or pay of Government" was a public servant be cause he was not an officer.
The case of Reg.
vs Ramajirav Jivbajirav(2) was cited in support of that contention and the learned Judges of the Calcutta High Court observed at page 346 as under: "The learned Judges in that case had to consider whether a lessee from Government was on the conditions of his lease a public servant and, in doing so, they considered generally the meaning of the term "officer".
It was there held that an officer means "some person employed to exercise, to some extent and in certain circumstances, a delegated function of Government.
He is either armed with some authority or representative character, or his duties are immediately auxiliary to those of some person who is so armed".
The meaning which we are asked to put on these words seems to us to be too narrow as applied to the present case.
The peon who has been convicted as a public servant is in service and pay of the Government, and he is attached to the office of the Superintendent of the Salt Department.
The exact (1) I.L.R. (2) XII Bom.
H.C.R. 1.
688 nature of his duties is not stated, because this objection was not taken at the trial.
, but we must take it that, from the nature of his appointment, it was his duty to carry out the orders of his official superior, who undoubtedly is a public servant, and in that capacity to assist the Superintendent in the performance of the public duties of his office.
In that sense he would be an officer of Government, although he might not possibly exercise "any delegated function of the Government".
Still his duties would be "immediately auxiliary to those of the Superintendent who is so armed".
We think that an "officer in the service or pay of Government" within the terms of section 21 of the Penal Code is one who is appointed to some office for the performance of some pulice duty.
In this sense the peon would come within section 21, cl. 9".
The true test, therefore, in order to determine whether a person is an officer of the Government, is: (1) whether he is in the service or pay of the Government, and (2) whether he is entrusted with the performance of any public duty.
If both these requirements are satisfied it matters not the least what is the nature of his office, whether the duties he is performing are of an exalted character or very humble indeed.
As has been stated in Bacon 's Abridgment at Vol.
6, page 2, in the article headed "Of the nature of an officer and the several kinds of officers": "The word "officium ' principally implies a duty, and in the next place, the charge of such duty; and that it is a rule that where one man 'bath to do with another 's affairs against his will, and without his leave, that this is an office, and he who is in it is an officer".
The next paragraph thereafter may also be referred to in this context: "There is, a difference between an office and an employment, every office being an employment; but there are employments which do not come under the denomination of offices; such as an agreement to, make hay, herd a flock, &c which differ widely from that of steward of a manor" &c. (Vide 12 Bombay High Court Reports at page 5).
689 This was the sense in which the decision in 12 Bombay High Court Reports 1, was understood by the learned Judges of the Lahore High Court in Ahad Shah vs Emperor(1) when they observed at page 157: "But it is not enough that a person should be in the pay or service of Government to Constitute him a public servant within the meaning of section 21 (ninthly), I.P.C. He must also be an "Officer".
That expression is not, of course, to be restricted to its colloquial meaning of a Commissioned or non Commissioned Officer; it means a functional or holder of some "officium" or office.
The office may be one of dignity or importance; it may equally be humble.
But whatever its nature, it is essential that.
the person holding the office, should have in some degree delegated to him certain functions of Government".
The question for consideration before the learned Judges of the Lahore High Court was whether a Quarter Master 's 'clerk was a public servant within the meaning of that expression in section 21, Indian Penal Code.
On the facts elicited before them the learned Judges came to the conclusion that the Quarter Master 's clerk as such was just a Babu and no more an officer than a labourer or menial employed and paid by Government to do public work (See Queen vs Nachimuttu(2).
if therefore on the facts of a particular case the Court comes to the conclusion that a person is not only in the service or pay of the 'Government but is also performing a public duty, he has delegated to him the functions of the Government or is in any event performing duties immediately ' auxiliary to those of some one who is an officer of the Government and is therefore 'an officer ' of the Government within the meaning of section 21(9), Indian Penal Code.
Applying this test to the facts of the case before us, we find that the appellant was a Class III servant and was employed as a metal examiner known as Chaser in the Railway Carriage Workshop.
He was working under the Works Manager who was certainly (1) A.I.R. 1918 Lah.
(2) I.L.R. 7 Madras 18, 690 an officer of the Government and the duties which he performed were immediately auxiliary to those of the Works Manager who, beside being an officer of the Government was also armed with some authority or representative character qua the Government.
The appellant was thus, even on a narrow interpretation of the dicta of West, J. in 12 Bombay High Court Reports 1, an officer in the service or pay of the Government performing as such a public duty entrusted to him by the Government and was therefore, a public servant within the meaning of section 21 of the Indian Penal Code.
This being the true legal position, this contention of the appellant also does not avail him and the first question must be answered against him.
The appellant was, therefore, an officer within the meaning of section 21(9) and therefore a public servant within the meaning of section 21, Indian Penal Code and being such public servant be fell within the definition of a public servant contained in section 2 of the Prevention of Corruption Act II of 1947.
He was, therefore, on the facts and circumstances of the case, rightly convicted under section 5(1) (d) of Act II of 1947.
His conviction and the sentence imposed upon him by the Courts below were therefore quite in order and this appeal must therefore stand dismissed.
"""

EXAMPLE_HEADNOTE_1 = """
The appellant was a Class III servant employed as a metal examiner, also called chaser, in the Railway Carriage Workshop at Ajmer.
He accepted a sum of Rs. 150 as illegal gratification for securing a job for some person.
He was charged under section 5(1)(d) of the Prevention of Corruption Act, 1947 (Act II of 1947).
The appellant contended that he was not an "officer" within the meaning of the term used in section 21(9) of the Indian Penal Code and so could not be a public servant for purposes of Act II of 1947.
It was found that the appellant was working under the Works Manager who was certainly an officer of the Government and the duties which he performed were immediately auxiliary to those of the Works Manager who was also armed with some authority or representative character qua the Government.
Held, that the appellant was an officer within the meaning of section 21(9) of the Indian Penal Code and, therefore, a public servant within the meaning of section 2 of Act 11 of 1947.
The true test in order to determine whether a person is an officer of the Government, is: " .
(1) whether he is in the service or pay of the Government and (2) whether he is entrusted with the performance of any,public duty.
The public duty may be either,a function of the Government delegated to him or may be a duty immediately auxiliary to that of someone who is an officer of the Government.
vs Ramajirav Jivbajirav ([1875] XII Bom.
H.C.R. 1), explained.
Nazamuddin V. Queen Empress, ([1900] I.L.R. and Ahad Shah vs Emperor (A.I.R. 1918 Leh. 152), relied on
"""

EXAMPLE_JUDGEMENT_2 = """
Civil Appeal No.169 of 1958.
Appeal by special leave from the decision dated June 29, 1955, of the Labour Appellate Tribunal of India, Bombay, in Appeal (Bombay) No. 245 of 1955, arising out of the Award dated July 14, 1955, of the Industrial Tribunal, Bombay, in Reference (I.T.) No. 100 of 1954.
5 34 C. K. Daphtary, Solicitor General of India, Purshottam Tricumdas and I. N. Shroff, for the appellant.
N. V. Phadke and K. R. Chaudhuri, for the respondents.
October 16.
The Judgment of the Court was delivered by GAJENDRAGADKAR J.
Are workmen entitled to the double benefit of a gratuity scheme as well as retrenchment compensation? That is the main question which falls to be considered in the present appeal.
The same question along with some other subsidiary points arises in some other appeals and so all of them have been grouped together and placed before us for disposal.
We propose to deal with the main point in the present appeal and discuss the other points arising in the other appeals separately.
This appeal by special leave arises from an industrial dispute between the Indian Hume Pipe Co. Ltd., Bombay (hereinafter called the appellant) and its workmen monthly rated including canteen boys employed under it (hereinafter called the respondents).
The dispute was in regard to the claim for gratuity made by the respondents and it was referred to the tribunal in these words: " Gratuity employees should be paid gratuity on the scale and the conditions prescribed in the industrial tribunal 's award in Ref.
(IT) No. 82 of 1950 dated August 13, 1951.
It should also be paid to those whose services have been terminated by the management after the termination of the aforesaid award.
" It appears that the respondents had raised an industrial dispute in 1950 which covered their claims for scale of pay, dearness allowance, provident fund and gratuity and it was referred to the adjudication of Mr. Thakore.
On this reference Mr. Thakore made his award on August 13, 1951, which inter alia provided for a scheme of gratuity.
Both the parties had gone in appeal against the said award but the appellate tribunal dismissed both the appeals and confirmed the award.
On June 2, 1953, notice was given by the respondents terminating the said award and making a fresh demand for gratuity at 35 a higher rate.
Conciliation proceedings were started but they failed; and so on July 1, 1954, the present reference was made.
Before the tribunal the employees urged that the State Government had no jurisdiction to confine their demand to the scheme of gratuity as framed by Mr. Thakore, and they urged the tribunal to consider their claim for a revision of the said scheme.
The tribunal held that its jurisdiction was limited by the terms of reference and it could not entertain any such plea; it also observed that even if it was open to the respondents to agitate for the revision of the said award there was not much chance of their succeeding in that demand.
The appellant opposed the scheme of gratuity framed by the earlier award and contended that no gratuity should be paid to the workmen who would be entitled to receive retrenchment compensation under section 25F of the (hereinafter called the Act).
This contention was negatived by the tribunal.
It held that the respondents were entitled to claim both gratuity and retrenchment compensation.
The tribunal then examined the financial position of the appellant and held that the gratuity scheme framed by the earlier award should be en forced subject to certain modifications specified by it.
This award was challenged by the appellant before the Labour Appellate Tribunal; and it was argued that the respondents were not entitled to the double benefit of the gratuity scheme and the statutory retrenchment compensation.
The appellate tribunal agreed with the view taken by the tribunal and rejected the appellant 's contention.
It also examined the financial position of the appellant and held that it saw no reason to interfere with the discretion exer cised by the tribunal in granting " the same gratuity to the workmen in the case of retrenchment as in other cases ".
Then the appellate tribunal considered the merits of the scheme sanctioned by the tribunal and made some changes and added one paragraph which had been included in the earlier award but had been omitted by the tribunal.
This paragraph dealt with the cases of persons retrenched after the date of 36 reference but before the award came into operation, and it directed that in the case of such persons no additional gratuity shall be paid if they have already received unemployment or retrenchment compensation in excess of the gratuity awarded above; in other cases the difference alone shall be paid.
It is against this award that the present appeal has been preferred.
On the contentions raised in the tribunals below, the principal point which calls for our decision is whether a scheme of gratuity can be framed by industrial tribunals for workmen who are entitled to the benefits of 25F of the Act.
This question has been frequently raised before industrial tribunals and has generally been answered in favour of the employees.
In dealing with this question it is important to bear in mind the true character of gratuity as distinguished from retrenchment compensation.
Gratuity is a kind of retirement benefit like the provident fund or pension.
At one time it was treated as payment gratuitously made by the employer to his employee at his pleasure, but as a result of a long series of decisions of industrial tribunals gratuity has now come to be regarded as a legitimate claim which workmen can make and which, in a proper case, can give rise to an industrial dispute.
Gratuity paid to workmen is intended to help them after retirement, whether the retirement is the result of the rules of superannuation or of physical disability.
The general principle underlying such gratuity schemes is that by their length of service workmen are entitled to claim a certain amount as a retrial benefit.
On the other hand retrenchment compensation is not a retirement benefit at all.
As the expression " retrenchment compensation" indicates it is compensation paid to a workman on his retrenchment and it is intended to give him some relief and to soften the rigor of hardship which retrenchment inevitably causes.
The retrenched workman is, suddenly and without his fault, thrown on the street and has to face the grim problem of unemployment.
At the com mencement of his employment a workman naturally expects and looks forward to security of service 37 spread over a long period; but retrenchment destroys his hopes and expectations.
The object of retrenchment compensation is to give partial protection to the retrenched employee and his family to enable them to tide over the hard period of unemployment.
Thus the concept on which grant of retrenchment compensation is based is essentially different from the concept on which gratuity is founded.
It is true that a retrenched workmen would by virtue of his retrenchment be entitled to claim retrenchment compensation in addition to gratuity; because industrial adjudication has generally taken the view that the payment of retrenchment compensation cannot affect the workmen 's claim for gratuity.
In, fact the whole object of granting retrenchment com pensation is to enable the workman to keep his gratuity safe and unused so that it may be available to him after his retirement.
Thus the object of granting retrenchment compensation to the employee is very different from the object which gratuity is intended to serve.
That is why on principle the two schemes are not at all irreconcilable nor even inconsistent ; they really complement each other; and so, on considerations of social justice there is no reason why both the claims should not be treated as legitimate.
The fact that they appear to constitute a double benefit does not affect their validity.
That is the view which industrial tribunals have generally taken in a large number of reported decisions on this point.
Let us now refer to some of these decisions and indicate very briefly the broad outlines of the development of industrial law on this subject.
Whenever industrial tribunals deal with the employees ' claim for gratuity they consider the financial position of the employer before granting the employees ' demand for framing a gratuity scheme; it is only if they are satisfied that the financial condition of the employer is satisfactory and the burden of the gratuity scheme can be borne by him that they proceed to frame schemes of gratuity and thereby secure for the employees the retirement benefit in the form of gratuity.
Though awards framing such schemes had been made for some 38 years before 1951, the question of framing a gratuity scheme was carefully examined by the Labour Appellate Tribunal in the case of The, Army and Navy Stores Ltd., Bombay, And Their Workmen (1).
The scheme framed in this case directed the payment of gratuity on the following scale: " (1) On the death of an employee while in the service of the company or on his becoming physically or mentally incapable of further service 'month 's salary or wages for each year of continuous service, to be paid to the disabled employee or, if he has died, to his heirs or legal represen tatives or assigns.
(2) On voluntary retirement or resignation of an employee after 15 years continuous service 1/2 month 's salary or wages for each year of continuous service.
(3) On termination of service by the company month 's salary or wages for each year of completed service.
" Under this scheme gratuity was not, however, payable to any employee dismissed for misconduct.
This scheme has been generally treated as a model scheme in all subsequent disputes about gratuity.
It also appears that the benefit of gratuity schemes has been generally given even to workmen whose services have been terminated and who have thereby become entitled to retrenchment compensation also.
In Bangalore Woollen, Cotton and Silk Mills Co. Ltd., And Binny Mills Labour Association (2) the Labour Appellate Tribunal gave permission to the company to retrench 179 workmen subject to the condition that the workmen sought to be retrenched shall be paid by way of retrenchment relief a sum equivalent to one month 's basic wage for every year of completed service in the company, and the basic wage on which such calculation is to be made shall be the last basic wage prior to the grant of this permission.
It also made it clear that the grant of such retrenchment relief shall not in any way tend to prejudice the issue (1) (2) 39 of a gratuity scheme which was before the adjudicator, and to which the adjudicator was directed to apply an altogether independent mind unaffected by the decision of the Labour Appellate Tribunal.
It may, however, be conceded that sometimes, though rarely; tribunals have thought it fit not to grant gratuity in cases of workmen whose services have been terminated on the ground that they would be entitled receive compensation under the Act.
But it is not disputed that this dissenting note has been struck only in a few cases (Vide Chemical, Industrial and Pharmaceutical Laboratory Ltd., And Their Workmen (1).
Speaking generally, subject to the capacity of the employer to pay, workmen have been given the benefit of both retrenchment compensation and gratuity by industrial awards prior to the enactment of section 25F of the Act.
This question was elaborately considered by the Labour Appellate Tribunal in the appeals against the award of All India Industrial Tribunal (Bank Disputes) where it has been held that the award of retrenchment compensation cannot adversely affect the claim for gratuity.
The two claims are made for entirely different reasons and in a proper case both the claims can be awarded.
The measure of compensation, however, varied from case to case, and the awards made in that behalf naturally were not always uniform.
But it does appear that the determination of the quantum of retrenchment compensation was generally linked with the period of the past service rendered by the retrenched workman.
In Rashtriya Mill Mazdoor Sangh and Gold Mohur Mills (2) the Labour Appellate Tribunal accepted the view that the quantum of compensation payable to retrenched workmen should be calculated at the rate of 10 days ' basic wages plus dearness allowance for each year of service; and it also held that no maximum limit should be put on this quantum, In the Bombay Gas Co. Ltd., And Their Workmen(3) a detailed scheme was framed for the computation of the retrenchment compensation.
Those who had completed a year 's service but less than three years ' service (1) (2) , (3) 40 got wages for 26 days with dearness allowance, and those who had completed three years of service or more got 26 days ' wages with dearness allowance for each year of service subject to a maximum of 104 days ' Wages with dearness allowance.
In The National Industrial Works And Their Workmen (1) a still more elaborate scheme was framed for.
determining the quantum of compensation.
Thus it would be seen that the result of industrial decisions was that workmen, were held entitled both to gratuity and compensation on retrenchment and the amount of retrenchment compensation was measured by reference to the period of service rendered by the retrenched employee.
It may, however, be stated that industrial decisions on the twin topics of gratuity and retrenchment compensation were not always uniform, and sometimes they disclosed an element of uncertainty and perhaps even ambiguity in their approach.
While this was the state of industrial decisions on this point, Ordinance V was promulgated on October 24, 1953.
By section 25E the Ordinance prescribed conditions precedent to retrenchment of workmen.
One of the conditions thus prescribed by section 25E(b) was that before a workman is retrenched he must be paid at the time of retrenchment, gratuity which shall be equivalent to 15 days ' average pay for every completed year of service or any part thereof in excess of six months.
This Ordinance was followed by Act 43 of 1953, which is deemed to have come into force on October 24, 1953.
It is by this amending Act that section 25F has been introduced in the Act.
Section 25F(b) is in the same terms as section 25E(b) of the Ordinance, except that for the word 'gratuity ' the expression "retrenchment compensation " has been substituted, We may incidentally mention the fact that in the statement of aims and objects of the Act it was observed that " in regard to retrenchment the bill provides that a workman who had been in continuous employment for not less than one year under the employer shall not be retrenched until he has been given one month 's notice in writing or one, (1) 41 month 's wages in lieu of such notice, and also a gratuity calculated at 15 days ' average pay for every completed year of service or any part thereof in excess of six months ".
The appellant 's case is that after section 25F was enacted there is no longer any scope for framing gratuity schemes in addition to the statuory retrenchment compensation for retrenched employees.
In support of this contention the appellant sought to rely on the fact that both in section 25E(b) of the Ordinance and the statement of aims and objects of the amending Act, the word ' gratuity ' has been used and not retrenchment compensation.
It is obvious that for construing section 25F the words used in the statement about the aims and objects of the Act are not relevant; and in regard to the use of the word ' gratuity ' in section 25E(b) of the Ordinance it is significant that the said word has been deliberately omitted and the words "retrenchment compensation" have been used in its place by section 25F.
Therefore it would not be possible to determine the character of the payment statutorily prescribed by section 25F by reference to the word 'gratuity ' used either by the Ordinance or in the statement about the aims and objects of the Act.
If we have to decide the character of the payment merely by the words used in describing it, then the words used section 25F are retrenchment compensation " and not gratuity.
But apart from the mere use of words there can be no doubt that section 25F is intended to provide compensation to retrenched workmen solely on account of the difficulties which they have to face on their retrenchment.
It is well known that at the time when the Ordinance was issued the problem of retrenchment had become widespread and acute and Legislature thought it necessary to step in and make a statutory provision for the payment of adequate retrenchment compensation.
Legislature knew that retrenchment compensation was being awarded by industrial tribunals; but it must have thought that in determining the amount of compensation the tribunals considered a variety of relevant factors with the result that there was no uniformity or certainty in the matter; and so 6 42 it decided to standardise the payment of compensation by prescribing a statutory rule in that behalf.
The enactment of section 25F thus merely standardises the payment of retrenchment compensation and nothing more.
If retrenchment compensation could be claimed by the employees in addition to gratuity prior to the enactment of section 25F there is no reason why a similar claim cannot be made by them subsequent to its enactment.
It is then urged that in determining the amount of compensation payable to a retrenched workman the length of his past service has been taken into account, and it is pointed out that schemes of gratuity also provide for payment of gratuity on similar considerations and adopt a similar measure.
As we have already pointed out, even before section 25F was enacted tribunals were adopting similar methods in determining the amount of retrenchment compensation, and so the mere fact that the length of the past service of the retrenched workman is made the basis for computing retrenchment compensation cannot clothe retrench ment compensation with the character of gratuity.
The claims for retrenchment compensation and gratuity proceed on different considerations and it would be impossible to bold that the grant of one excludes the claim or grant of the other.
It is true that a retrenched workman would get both the retrenchment compensation and gratuity, and in a sense, on his retrenchment he would get more than what other workmen with corresponding length of service would get on their retirement; but it must be remembered that the retrenched workman gets compensation because involuntarily he has been forced to face unemployment, and it is to enable him to tide over the period of unemployment that retrenchment compensation is paid to him.
So, on the general contention raised before us that the employees are not entitled to claim the double benefit of gratuity and retrenchment compensation there can be only one answer,and that is that there is no conflict between the two claims, and industrial tribunals are right in recognising that both claims can be entertained and 43 granted, and reasonable gratuity schemes can and should be framed even after the enactment of section 25F in the Act.
In this connection it would be relevant to refer to the definition of wages under section 2(rr) of the Act inasmuch as it excludes any gratuity payable on the termination of the employee 's service.
This shows that Legislature was aware that gratuity can be claimed by employees and is often awarded to them.
If Legislature had intended that the statutory retrenchment compensation provided for by section 25F should affect the employees ' claim for gratuity it would have expressly made a suitable provision in that behalf.
Legislature makes such provisions when it thinks necessary to do so.
Section 17 of the Employees ' Provident Funds Act, 1952 (Act 19 of 1952), for instance, confers on the appropriate Government power to exempt from the operation of all or any of the provisions of the scheme, establishments which have already introduced provident fund benefits which, on the whole, are not less favourable to the employees than the benefits provided under this Act.
In the absence of any such provision in the it would be unreasonable to hold that the mere enactment of section 25F either ousts the jurisdiction of industrial tribunals to entertain claims for gratuity schemes or makes it improper or unjust to frame such schemes for all employees including those who are retrenched.
So far we have dealt with the general question as it arose on the contentions of the parties; but in fairness we must add that the learned Solicitor General conceded that he could not urge that, as a matter of law, the point raised by his client should be answered in his favour.
He, however, strenuously urged that in framing gratuity schemes industrial tribunals should make appropriate provision for giving gratuity to retrenched workmen on a basis different from that on which gratuity to other workmen is calculated.
The argument is that since the retrenched workmen get statutory compensation on a very liberal scale they should not get gratuity at the rates fixed by the scheme for other workmen.
They may and should get gratuity 44 but at a lesser rate and on less generous terms and conditions.
Indeed he suggested that we should make suitable amendments in the gratuity scheme framed by the appellate tribunal in that behalf.
We do not think we can accede to this request.
Whether or not a twofold scheme of gratuity should be framed, one applicable to retrenched workmen and the other to the rest, is a matter which may, if necessary, be raised before the tribunal in a proper case.
Besides it may be pertinent to observe that the question as presented in this form is not one of general importance, for in the present state of our economy which has received and is receiving the stimulus of national plans, our industries may not have to face the problem of retrenchment on an appreciable or extensive scale; but apart from this consideration we cannot entertain or decide the point raised by the learned Solicitor General in an appeal under article 136.
Before we part with this appeal, we ought to refer to another aspect of the matter which our present decision does not consider or decide.
It is likely that gratuity schemes framed by consent or by awards may provide for payment of compensation to retrenched workmen either in lieu of or in addition to gratuity ; in such cases the question as to whether the retrenched workmen can claim the benefit of such a scheme in addition to the retrenchment compensation under section 25F would depend on the construction of the material terms of the relevant scheme considered in the light of the provisions of section 25F of the Act.
In the present appeal we are not called upon to consider such a question.
Therefore, our decision has and can have no reference to cases which would fall to be decided under section 25F of the Act.
In the result the appeal fails and is dismissed with costs.
Appeal dismissed.
"""

EXAMPLE_HEADNOTE_2 = """
Section 25F(b) of the , provided: "No workman employed in any industry who has been in continuous service for not less than one year under an employer shall be retrenched by that employer until . (b) the workman has been paid, at the time of retrenchment, compensation which shall be equivalent to fifteen days ' average pay for every completed year of service or any part thereof in excess of six months .
" The dispute between the appellant company and its workmen related to the claim for gratuity made by the latter and it was the appellant 's contention that in the scheme of gratuity framed by the Tribunal no gratuity should be paid to workmen who would be entitled to receive retrenchment compensation under section 25F of the .
Before section 25F was introduced in the Act by Act 43 of 1953, workmen were given the benefit of both retrenchment compensation and gratuity by industrial awards, but the decisions were not always uniform.
Ordinance V was promulgated on October 24, 1953, by section 25E(b) of which it was provided that before a workman was retrenched he must be paid at the time of retrenchment gratuity which shall be equivalent to 15 days ' average pay for every completed year of service or any part thereof in excess of six months.
The 33 Ordinance was followed by Act 43 of 1953, which was deemed to have come into force on October 24, 1953; and in the statement of aims and objects of the Act it was said " that a workman . shall not be retrenched until he has been given one month 's notice in writing or one month 's wages in lieu of such notice, and also a gratuity . ".
Section 25(F)(b) of the Act was in the same terms as section 25E(b) of the Ordinance,, except that for the word CC gratuity " the expression " retrenchment compensation " was substituted.
The appellant 's case was that after section 25F was enacted there was no longer any scope for framing gratuity schemes in addition to the statutory retrenchment compensation for retrenched employees on the grounds (1) that both in section 25E(b) of the Ordinance and the statement of aims and objects of the amending Act, the word "gratuity" had been used and not retrenchment compensation, (2) that in determining the amount of compensation payable to a retrenched workman the length of his past service had been taken into account, and schemes of gratuity also provide for payment of gratuity on similar considerations and adopt a similar measure, and (3) that a retrenched workman would get both retrenchment compensation and gratuity and so get more than what other workmen with corresponding length of service would get on their retirement.
Held: (1) that for construing S, 25F of the , the words used in the statement about the aims and objects of the Act are not relevant and that the character of the payment prescribed by the section could only be determined by the expression " retrenchment compensation used therein ; and, (2) that in the absence of any provision in the excluding the claim or grant of gratuity the mere enactment of section 25F cannot oust the jurisdiction of industrial tribunals to entertain claims for gratuity schemes or make it improper or unjust to frame such schemes for all employees including those who are retrenched.
The object of granting retrenchment compensation is to enable the workman who is given partial protection to tide over the period of unemployment, and to keep his gratuity safe and unused so that it may be available to him after his retirement.
The two claims complement each other, and the fact that they appear to constitute a double benefit cannot affect their validity.
"""

In [6]:
RUBRIC_PROMPT = f"""
You are a professor at a law school and are in charge of grading headnotes/summaries students have written for court judgements.
Case judgments represent rulings issued by judges in judicial proceedings, often comprising extensive transcripts spanning dozens of pages.
An excellent headnote captures the core of the judgment properly, succinctly, and completely.
Your job is to judge and grade the written headnotes, so be as strict as possible.

This is the rubric:
- PAST TENSE: The headnote is written in the past tense.
- METADATA: The headnote includes the case name, judgement number, court, judge(s), and date of judgement.
- INTRODUCTION: The headnote introduction describes the field of law that the case deals with, and does not directly jump into the case details.
- KEY FACTS: The headnote includes who filed the case, why the case was filed, and what remedy the filer wants.
- KEY ARGUMENTS: The headnote includes both sides' submissions and reasons to support their position.
- RELEVANT LAWS: The headnote references relevant legislation and concepts to support the case judgement.
- CONCLUSION: The headnote includes the case's conclusion and procedural disposition (ex: permitted, dismissed, reversed, remanded, affirmed, etc.).

Your job is to read the judgement and headnote provided, then go through the rubric items one by one.
Mark each rubric item as TRUE or FALSE.
Output each rubric item and what you assigned it.
Do not introduce any other rubric items.
Do not output anything else except for the 7 bullet points corresponding to rubric items - no more, no less.
Your output should be final, and no more changes should be made.

Format of your output:
- PAST TENSE: TRUE/FALSE
- METADATA: TRUE/FALSE
- INTRODUCTION: TRUE/FALSE
- KEY FACTS: TRUE/FALSE
- KEY ARGUMENTS: TRUE/FALSE
- RELEVANT LAWS: TRUE/FALSE
- CONCLUSION: TRUE/FALSE

Example Judgement: {EXAMPLE_JUDGEMENT_1}
Example Headnote: {EXAMPLE_HEADNOTE_1}
Example Output:
- PAST TENSE: FALSE
- METADATA: TRUE
- INTRODUCTION: FALSE
- KEY FACTS: TRUE
- KEY ARGUMENTS: TRUE
- RELEVANT LAWS: TRUE
- CONCLUSION: FALSE

Example Judgement: {EXAMPLE_JUDGEMENT_2}
Example Headnote: {EXAMPLE_HEADNOTE_2}
Example Output:
- PAST TENSE: TRUE
- METADATA: FALSE
- INTRODUCTION: TRUE
- KEY FACTS: TRUE
- KEY ARGUMENTS: TRUE
- RELEVANT LAWS: TRUE
- CONCLUSION: FALSE
"""

## Import and Split the Dataset

In [8]:
original_df = pd.read_csv(INPUT_FILE, index_col=0)
original_df.head(5)

,judgement,headnote
0,Special Leave Petition Nos.\n823 24 of 1990.\n...,Petitioners ' lands were acquired by the respo...
1,ivil Appeal No. 4649 of 1989.\nFrom the Judgme...,Pursuant to a scheme enacted for the benefit o...
2,"Appeals, Nos. 275 276 of 1963.\nAppeals by spe...","By section 25 (4) of the Income tax Act, ""Wher..."
3,No. 7338 of 1981.\n(Under Article 32 of the Co...,Fundamental Rule 56(j) confers power on the ap...
4,(C) No. 677 of 1988.\n(Under Article 32 of the...,The Lt. Governor of Delhi amended the Delhi Po...


In [9]:
# check how many records have been processed so we don't do them again
# helpful if the file crashes/stops for any reason
if os.path.exists(OUTPUT_FILE):
  num_processed = len(pd.read_csv(OUTPUT_FILE))
else:
  num_processed = 0

original_df_chunks = np.array_split(original_df, len(original_df)/CSV_CHUNK_SIZE)
original_df_chunks = original_df_chunks[(num_processed // CSV_CHUNK_SIZE):]
print(f'Number of records left to process: {len(original_df_chunks) * CSV_CHUNK_SIZE}')

Number of records left to process: 860


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


## Load the Model

In [10]:
# load the tokenizer because we need access to the pad/eos token ID to suppress runtime warnings later on
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

pipe = pipeline(
    "text-generation",
    model=MODEL_NAME,
    device_map="auto",
    # using this precision since we are running on A100 GPU
    torch_dtype=torch.bfloat16,
    return_full_text=True,
    model_kwargs={
        "quantization_config": BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.bfloat16
        )
    },
    pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


## Loop through the Judgements

In [12]:
def append_to_csv(entries):
    if not os.path.exists(OUTPUT_FILE):
      entries = [["judgement", "headnote", "response", "cleaned_response"]] + entries
    with open(OUTPUT_FILE, 'a') as file:
        writer = csv.writer(file)
        writer.writerows(entries)

In [13]:
# classify the response as GREAT, MEDIUM, POOR, or INVALID RESPONSE based on the model output
def clean_response(headnote, response):
  response = response.lower()
  overall_score = 0
  rubric_counts = dict()

  for rubric_item in ["past tense", "metadata", "introduction", "key facts", "key arguments", "relevant laws", "conclusion"]:
    for condition in ["true", "false"]:
      rubric_counts[f'{rubric_item} {condition}'] = response.count(f'{rubric_item}: {condition}')

    # sometimes Llama 3 repeats itself to pad up to the new token count, so treat extra counts just as one
    if rubric_counts[f'{rubric_item} true'] > rubric_counts[f'{rubric_item} false']:
      overall_score += 1

  if overall_score == 0:
    return 'INVALID RESPONSE'

  word_count = len(headnote.split(' '))
  if word_count >= 300 and word_count <= 800:
    overall_score += 1

  if overall_score > 6:
    return 'GREAT'
  elif overall_score > 3:
    return 'MEDIUM'
  else:
    return 'POOR'

In [14]:
# reconstruct the prompt components from the model response
def split_response(response_string):
  response_without_prompt = response_string.split('\n\nJudgement:')[1]
  judgement_split = response_without_prompt.split('\n\nHeadnote:')
  judgement = judgement_split[0].strip()

  headnote_split = judgement_split[1].split('\n\nYour Output:')
  headnote = headnote_split[0].strip()
  response = headnote_split[1].strip()
  cleaned_response = clean_response(headnote, response)

  return judgement, headnote, response, cleaned_response


In [15]:
# yield rows to help parallelize + utilize GPU efficiently
def generator():
  for df in original_df_chunks:
    for _, row in df.iterrows():
      judgement, headnote = row["judgement"], row["headnote"]
      prompt = f'{RUBRIC_PROMPT}\n\nJudgement: {judgement}\n\nHeadnote: {headnote}\n\nYour Output:'
      yield prompt

In [16]:
headnotes_graded = 0
responses_seen = []

for full_response in tqdm(pipe(generator(), max_new_tokens=300), total=len(original_df_chunks) * CSV_CHUNK_SIZE):
  full_response = full_response[0]['generated_text']
  judgement, headnote, response, cleaned_response = split_response(full_response)
  headnotes_graded += 1
  responses_seen.append([judgement, headnote, response, cleaned_response])

  if headnotes_graded == CSV_CHUNK_SIZE:
    append_to_csv(responses_seen)
    headnotes_graded = 0
    responses_seen = []

  0%|          | 0/860 [00:00<?, ?it/s]